### 1.0) API?

API é a ponte entre softwares. Consiste em um fornecimento de capacidades para outros aplicativos ou softwares.

A interface, que seria o próprio site do chat GPT, exige uma interação manual e um pouco limitada para resolução. Já a API com a conexão a outros softwares de programação, consegue extrair mais profundamente suas especificidades.

### 1.1) Aprendendo a fazer perguntas e receber respostas do chat em python

In [1]:
### importando a api do chat
import openai

Agora, precisamos conectar a nossa chave de acesso no própio site da openAI. Como a chave é uma particularidade do usuário, vamos salva-la em um arquivo chamado de arquivo .ENV

Esse arquivo é como um lugar secreto disponivel apenas para o usuário, sem dar permissão de outras pessoas terem acesso

Para acessa-lo, vamos fazer o seguinte:

In [2]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [3]:
client = openai.Client()

Explicando os dois códigos acima. O load_dotenv carrega a chave que colocamos lá no arquivo .env
quando rodamos o código openai.Client ele consegue acessar porque a chave já tinha sido carregada.

Caso não fizessemos isso, teríamos que colocar a chave dentro do comando Client()

#### Como enviar perguntas e recebe-las?

Agora vamos para a parte que interessa

In [8]:
#criando a pergunta

mensagem = [{'role':'user', 'content': 'o que é uma maçã em 5 palavras?'}]

resposta = client.chat.completions.create(
    messages=mensagem,
    model = 'gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0
)

resposta

ChatCompletion(id='chatcmpl-At1tXRzvlx77smlYDvCBk1rik2QHR', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Fruta redonda e saborosa.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1737678687, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=20, total_tokens=30, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

Explicando brevemente, Primeiro, a mensagem que criamos fizemos uma lista de um dicionário. Isso porque, se você analisar uma conversa com o chat, sempres temos uma pergunta (dicionário com o "role"(cargo) sendo user) e a resposta do chat, que seria outro dicionário com o "role" sendo assistant. A lista nada mais é do que o conjunto de dicionários sendo este as perguntas e as respostas.

se você reparar, o comando "resposta" que criamos, contém como parametrôs a mensagem, o modelo que queremos que gere a resposta, o número de tokens para a resposta e a temperatura que seria o quão "viajado" é a resposta

além disso, quando rodamos "resposta", obtemos um objeto com várias atribuições e métodos. Para obter a resposta em si, temos que ir """cavando""" os atributos até chegar nela

In [9]:
resposta.choices[0].message.content

'Fruta redonda e saborosa.'

Agora, lembra que comentamos sobre o comando mensagem ser uma lista de perguntas e respostas? Logo, como obtivemos a resposta do chat, podemos adiciona-la em nosso comando:


In [10]:
mensagem.append({'role':'assistant', 'content': resposta.choices[0].message.content})

In [11]:
mensagem

[{'role': 'user', 'content': 'o que é uma maçã em 5 palavras?'},
 {'role': 'assistant', 'content': 'Fruta redonda e saborosa.'}]

Podemos adicionar outras perguntas e completar com outras respostas. Basta continuar com esse mesmo raciocínio. E se fizermos uma função que execute isso automaticamente?
Então, bora lá

#### Função que printa a resposta e faz o append automaticamente

In [13]:
### Criando a função

def gerador_texto(mensagens, model = 'gpt-3.5-turbo-0125', max_tokens =1000, temperature=0):
    
    resposta = client.chat.completions.create(
        messages = mensagens,
        model=model,
        max_tokens=max_tokens,
        temperature=temperature,
    )
    print(resposta.choices[0].message.content)
    mensagens.append({'role':'assistant', 'content':resposta.choices[0].message.content})
    
    return mensagens

In [14]:
teste = [{'role':'user', 'content':'me explique a lei da relatividade'}]

gerador_texto(teste, max_tokens= 500)


A teoria da relatividade é uma das teorias mais importantes da física moderna, proposta por Albert Einstein no início do século XX. Ela consiste em duas partes principais: a relatividade restrita e a relatividade geral.

A relatividade restrita afirma que as leis da física são as mesmas para todos os observadores que se movem em velocidades constantes em relação uns aos outros. Isso significa que não existe um referencial absoluto no universo, e que o tempo, o espaço e a massa são relativos à velocidade do observador.

Já a relatividade geral afirma que a gravidade não é uma força como as outras, mas sim uma curvatura do espaço-tempo causada pela presença de massa e energia. Isso significa que objetos em movimento seguem trajetórias curvas no espaço-tempo, devido à influência da gravidade.

Em resumo, a teoria da relatividade de Einstein revolucionou nossa compreensão do universo, mostrando que o tempo, o espaço e a gravidade são interligados de maneiras complexas e surpreendentes. Ela

[{'role': 'user', 'content': 'me explique a lei da relatividade'},
 {'role': 'assistant',
  'content': 'A teoria da relatividade é uma das teorias mais importantes da física moderna, proposta por Albert Einstein no início do século XX. Ela consiste em duas partes principais: a relatividade restrita e a relatividade geral.\n\nA relatividade restrita afirma que as leis da física são as mesmas para todos os observadores que se movem em velocidades constantes em relação uns aos outros. Isso significa que não existe um referencial absoluto no universo, e que o tempo, o espaço e a massa são relativos à velocidade do observador.\n\nJá a relatividade geral afirma que a gravidade não é uma força como as outras, mas sim uma curvatura do espaço-tempo causada pela presença de massa e energia. Isso significa que objetos em movimento seguem trajetórias curvas no espaço-tempo, devido à influência da gravidade.\n\nEm resumo, a teoria da relatividade de Einstein revolucionou nossa compreensão do univer

### 1.2 Explorando o objeto resposta.

Nós utilizamos apenas o conteúdo da resposta como "significante". Mas podemos obter outros outputs desse objeto como

1. Uso de tokens

In [15]:
mensagem = [{'role':'user', 'content': 'o que é uma maçã em 5 palavras?'}]

resposta = client.chat.completions.create(
    messages=mensagem,
    model = 'gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0
)

#### USO DE TOKENS
resposta.usage

CompletionUsage(completion_tokens=10, prompt_tokens=20, total_tokens=30, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

2. Comando **"model_dump"**

Esse comando nos oferece a resposta naquele formato de dicionário com o role : content etc..

In [16]:
resposta.choices[0].message.model_dump()

{'content': 'Fruta redonda e saborosa.',
 'refusal': None,
 'role': 'assistant',
 'audio': None,
 'function_call': None,
 'tool_calls': None}

Repare que tem outros elementos nesse dicionário que veremos mais a frente, mas que estão classificados como None.

Se quisermos tirar o none, podemos usar:

In [17]:
resposta.choices[0].message.model_dump(exclude_none=True)

{'content': 'Fruta redonda e saborosa.', 'role': 'assistant'}

### 1.3) Recebendo resposta em tempo real 

Bem, se você analisar, quando fizermos uma pergunta e rodar a resposta, pode ser que dependendo da complexidade da resposta ele demore a responder. Por isso, podemos configurar para que ele gere as respostas em tempo real.

Com o argumento **STREAM** = **true**, transformamos a nossa resposta de objeto à um iterador de chunks

O que são chunks? Basicamente são pedaços de texto.

Observe então:

In [25]:
mensagem = [{'role':'user', 'content': 'me fale sobre a teoria da relatividade'}]

resposta = client.chat.completions.create(
    messages=mensagem,
    model = 'gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0,
    stream = True
)

In [20]:
## virou um iterador de chunks
resposta

Agora, como todo iterador, vamos colocar em um for que apenas printa:

In [21]:
for chunks in resposta:
    print(chunks)

ChatCompletionChunk(id='chatcmpl-At391Q4BwT1lvHuUw6LblNerEMGkQ', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1737683491, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', service_tier='default', system_fingerprint=None, usage=None)
ChatCompletionChunk(id='chatcmpl-At391Q4BwT1lvHuUw6LblNerEMGkQ', choices=[Choice(delta=ChoiceDelta(content='A', function_call=None, refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1737683491, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', service_tier='default', system_fingerprint=None, usage=None)
ChatCompletionChunk(id='chatcmpl-At391Q4BwT1lvHuUw6LblNerEMGkQ', choices=[Choice(delta=ChoiceDelta(content=' te', function_call=None, refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1737683491, model='gpt-3.5-turbo-0125',

Repare, leia os choices e o atributo "delta" depois "content". São vários pedaços de texto.

Ok, então não basta printar o chunk. Pois o chunk seria o próprio objeto resposta mas apenas para um chunk!

In [28]:
mensagem = [{'role':'user', 'content': 'me fale sobre a teoria da relatividade'}]

resposta = client.chat.completions.create(
    messages=mensagem,
    model = 'gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0,
    stream = True
)


#### obtendo o texto em si
for chunks in resposta:
    pedaco_de_texto = chunks.choices[0].delta.content
    if pedaco_de_texto != None: ## o último content tem esse "none" não sei porque
        print(pedaco_de_texto, end = '')


A teoria da relatividade é uma das teorias mais importantes da física moderna, proposta por Albert Einstein no início do século XX. Ela é composta por duas partes: a teoria da relatividade restrita e a teoria da relatividade geral.

A teoria da relatividade restrita, proposta por Einstein em 1905, postula que as leis da física são as mesmas para todos os observadores que se movem em velocidades constantes em relação uns aos outros. Isso significa que não existe um referencial absoluto no universo e que o tempo, o espaço e a massa são relativos, dependendo da velocidade do observador.

Já a teoria da relatividade geral, proposta por Einstein em 1915, descreve a gravidade como a curvatura do espaço-tempo causada pela presença de massa e energia. Segundo essa teoria, corpos massivos como planetas e estrelas curvam o espaço ao seu redor, criando o que chamamos de campo gravitacional.

A teoria da relatividade revolucionou a forma como entendemos o universo e teve importantes consequências 

Beleza, e agora, se eu quiser salvar esse texto em uma variável? Vamos lá!

In [32]:
mensagem = [{'role':'user', 'content': 'me fale sobre a teoria da relatividade'}]

resposta = client.chat.completions.create(
    messages=mensagem,
    model = 'gpt-3.5-turbo-0125',
    max_tokens=4000,
    temperature=0,
    stream = True
)


#### obtendo o texto em si
Rfinal = ''
for chunks in resposta:
    pedaco_de_texto = chunks.choices[0].delta.content
    if pedaco_de_texto != None: ## o último content tem esse "none" não sei porque
        Rfinal += pedaco_de_texto
        print(pedaco_de_texto, end = '')


A teoria da relatividade é uma das teorias mais importantes da física moderna, proposta por Albert Einstein no início do século XX. Ela é composta por duas partes: a teoria da relatividade restrita e a teoria da relatividade geral.

A teoria da relatividade restrita, proposta por Einstein em 1905, estabelece que as leis da física são as mesmas para todos os observadores que se movem a uma velocidade constante em relação uns aos outros. Isso significa que não existe um referencial absoluto no universo e que o tempo, o espaço e a massa são relativos, dependendo da velocidade do observador.

Já a teoria da relatividade geral, proposta por Einstein em 1915, descreve a gravidade como a curvatura do espaço-tempo causada pela presença de massa e energia. Segundo essa teoria, corpos massivos como planetas e estrelas curvam o espaço ao seu redor, o que faz com que outros corpos se movam ao longo dessa curvatura, seguindo trajetórias curvas.

A teoria da relatividade revolucionou a física ao pro

In [30]:
Rfinal

'A teoria da relatividade é uma das teorias mais importantes da física moderna, proposta por Albert Einstein no início do século XX. Ela é composta por duas partes: a teoria da relatividade restrita e a teoria da relatividade geral.\n\nA teoria da relatividade restrita, proposta por Einstein em 1905, revolucionou a forma como entendemos o espaço e o tempo. Ela postula que as leis da física são as mesmas para todos os observadores em movimento uniforme, independentemente da velocidade relativa entre eles. Isso significa que não existe um "tempo absoluto" ou um "espaço absoluto", mas sim que o tempo e o espaço são relativos e dependem do observador.\n\nJá a teoria da relatividade geral, proposta por Einstein em 1915, descreve a gravidade como a curvatura do espaço-tempo causada pela presença de massa e energia. Segundo essa teoria, corpos massivos como planetas e estrelas curvam o espaço ao seu redor, o que faz com que outros corpos se movam ao longo de trajetórias curvas.\n\nA teoria da